# NLP - HW5
### Miguel Bonilla

In [2]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

- [1. Compile a List of Reviews](#1.-Compile-a-List-of-Reviews)
- [2. Extract Noun Phrase Chunks](#2.-Extract-Noun-Phrase-Chunks)
- [3. Output All the Chunks](#3.-Output-All-the-Chunks)

### 1. Compile a List of Reviews

Using BeautifulSoup to crawl through the featured 25 reviews for each of the films, for a total of 100 reviews. The four movies are: "The Thing (1982)", "A Quiet Place", "Alien Covenant", and "The Shining". Each of these has a mix of positive and negative reviews, though the distribution of these depends on the 'quality' of the film. 

In [3]:
### assign headers since IMDB rejects the requests without it
headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 Edg/110.0.1587.50'}

In [4]:
## The Thing (1982)
url1 = 'https://www.imdb.com/title/tt0084787/reviews/?ref_=tt_ov_rt'
## A Quiet Place
url2 = 'https://www.imdb.com/title/tt6644200/reviews/?ref_=adv_li_tt'
## Alien Covenant
url3 = 'https://www.imdb.com/title/tt2316204/reviews/?ref_=adv_li_tt'
## The Shining
url4 = 'https://www.imdb.com/title/tt0081505/reviews/?ref_=fn_al_tt_1'

#movie list
movies = ['The_Thing','A_Quiet_Place','Alien_Covenant','The_Shining']
#url list
urls = [url1, url2, url3, url4]

In [5]:
def get_links(movielist,urllist):
    refs = []
    index = []
    for i in range(len(movielist)):
        movie = get(urllist[i],headers=headers)
        movie_soup = BeautifulSoup(movie.content,'html.parser')
        container = movie_soup.find_all(class_='title')
        for j in range(len(container)):
            refs.append('https://www.imdb.com'+ container[j]['href'])
            index.append(movielist[i])
    return(pd.DataFrame({'movie':index,'url':refs}))
        

In [6]:
review_links = get_links(movies,urls)
review_links

,movie,url
0,The_Thing,https://www.imdb.com/review/rw0197822/?ref_=tt...
1,The_Thing,https://www.imdb.com/review/rw3346521/?ref_=tt...
2,The_Thing,https://www.imdb.com/review/rw1833451/?ref_=tt...
3,The_Thing,https://www.imdb.com/review/rw6379386/?ref_=tt...
4,The_Thing,https://www.imdb.com/review/rw0197779/?ref_=tt...
...,...,...
95,The_Shining,https://www.imdb.com/review/rw0179869/?ref_=tt...
96,The_Shining,https://www.imdb.com/review/rw7669173/?ref_=tt...
97,The_Shining,https://www.imdb.com/review/rw0180168/?ref_=tt...
98,The_Shining,https://www.imdb.com/review/rw2504629/?ref_=tt...


In [7]:
review_links.url[1]

'https://www.imdb.com/review/rw3346521/?ref_=tt_urv'

### 2. Extract Noun Phrase Chunks

#### a. Grab the review text from each link

In [8]:
review = get(review_links.url[1],headers)

In [9]:
review_soup =BeautifulSoup(review.content,'html.parser')

In [10]:
container = review_soup.find(class_='text show-more__control').stripped_strings

In [14]:
string_sav =[]
for string in review_soup.find(class_='text show-more__control').stripped_strings:
    sent = sent_tokenize(string)
    print(sent)

['A classic film.', 'John Carpenter\'s "The Thing" is one of the most entertaining horror films ever made - fast, clever and purely exciting from start to finish.', 'It is one of my personal favorite horror movies.', 'This is how all movies of the genre should be made.', 'Set on an isolated base in Antarctica, this version seems almost to pick up where the original version (The Thing From Another World) left off.', 'The American scientists discover a decimated Norwegian base some miles distant.', 'Everyone is dead, and only the half charred remains of some unidentifiable thing left to smolder outside the compound might offer any answers to what may have happened.', 'The Thing is brought back to the American base and, too late, the scientists realize that it is alive and lethal.', 'The Thing thaws out and is off, not only killing anyone and anything that crosses Its path, but also absorbing them, making Itself into whoever and whatever it wants.', 'The film then turns into a brilliant p

### 3. Output All the Chunks